In [1]:
import torch
import sys
import importlib
import models.tsg_loss as tsg_loss
import models
from models import tsg_centroid_module, tsg_seg_module
from models.generator import CenterPointGenerator
import os
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [3]:
centroid_model = models.tsg_centroid_module.get_model()
centroid_model.cuda()
centroid_model.load_state_dict(torch.load("pretrained_centroid_model.h5"))
centroid_model.eval()

point_loader = DataLoader(CenterPointGenerator(), batch_size=2)

seg_model = tsg_seg_module.get_model()
seg_model.load_state_dict(torch.load("model_seg"))
seg_model.cuda()
seg_model.eval()


get_model(
  (sa1_1): PointNetSetAbstractionMsg(
    (conv_blocks): ModuleList(
      (0): ModuleList(
        (0): Conv2d(9, 32, kernel_size=(1, 1), stride=(1, 1))
        (1): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
      )
      (1): ModuleList(
        (0): Conv2d(9, 32, kernel_size=(1, 1), stride=(1, 1))
        (1): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
      )
    )
    (bn_blocks): ModuleList(
      (0): ModuleList(
        (0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): ModuleList(
        (0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
  )
  (sa2_1): PointNetSetAbstractionMsg(
    (conv_blocks): ModuleList(
      (0): ModuleList(
        (0): Conv2d(67, 64, kernel_si

In [83]:
for item in point_loader:
    points = item[0].cuda()
    centroids = item[1].cuda()
    output = centroid_model(points)
    seg_label = item[2].cuda()


In [84]:
#첫번쨰 아웃풋 cpu
output_offset_cpu = output[4].cpu().detach().numpy()[0,:].T
output_xyz_cpu = output[3].cpu().detach().numpy()[0,:].T
cen_cpu = centroids.cpu().detach().numpy()[0,:].T
output_dist_cpu = output[5].cpu().detach().numpy()[0,:].T
output_input_xyz_cpu = output[2].cpu().detach().numpy()[0,:].T

In [85]:
#이건 크롭 시각화
import torch
import models.tsg_utils as utils
import importlib
importlib.reload(utils)

#o3d.visualization.draw_geometries([np_to_pcd(output_offset_cpu+output_xyz_cpu, color=[0,1,0]), np_to_pcd(cen_cpu)], mesh_show_back_face=False,mesh_show_wireframe=True)
#o3d.visualization.draw_geometries([np_to_pcd(output_xyz_cpu, color=[0,1,0]), np_to_pcd(cen_cpu)], mesh_show_back_face=False,mesh_show_wireframe=True)
#o3d.visualization.draw_geometries([np_to_pcd(output_xyz_cpu[output_gt_dist_cpu.reshape(256)<0.02], color=[0,1,0]), np_to_pcd(cen_cpu)], mesh_show_back_face=False,mesh_show_wireframe=True)
#o3d.visualization.draw_geometries([np_to_pcd(output_xyz_cpu[output_dist_cpu.reshape(256)<0.02], color=[0,1,0]), np_to_pcd(cen_cpu)], mesh_show_back_face=False,mesh_show_wireframe=True)
#sampled_db_scan = utils.dbscan_pc(output[3], output[4], output[5])
#o3d.visualization.draw_geometries([np_to_pcd(sampled_db_scan[0], color=[0,1,0]), np_to_pcd(cen_cpu)], mesh_show_back_face=False,mesh_show_wireframe=True)

for i in range(15):
    gt_bin_label = get_gt_mask_labels(pred_seg[0], pred_seg[1],cropped_gt_labels)
    test_cropped = cropped_coords.cpu().detach().numpy()[i].T
    test_gt_label = gt_bin_label.cpu().detach().numpy()[i].T.reshape(-1,1)
    test_cropped = np.concatenate([test_cropped, test_gt_label],axis=1)

    test_cropped_2 = cropped_coords.cpu().detach().numpy()[i].T
    test_gt_label_2 = gt_bin_label.cpu().detach().numpy()[i].T.reshape(-1,1)
    test_cropped_2 = np.concatenate([test_cropped_2, test_gt_label_2],axis=1)
    #o3d.visualization.draw_geometries([np_to_pcd(test_cropped, color=[0,1,0]), np_to_pcd(cen_cpu)], mesh_show_back_face=False,mesh_show_wireframe=True)
    o3d.visualization.draw_geometries([np_to_by_label(test_cropped, axis=3), np_to_by_label(test_cropped_2, axis=3),np_to_pcd(cen_cpu)], mesh_show_back_face=False,mesh_show_wireframe=True)


NameError: name 'get_gt_mask_labels' is not defined

In [16]:
#두번째 아웃풋을 위한 정리
import torch
import models.tsg_utils as utils
import importlib
importlib.reload(utils)
sampled_db_scan = utils.dbscan_pc(output[3], output[4], output[5])
nearest_n = utils.get_nearest_neighbor_idx(output[2], sampled_db_scan, 1024)
cropped_coords, _ = utils.get_nearest_neighbor_points_with_centroids(output[2], nearest_n, sampled_db_scan)
cropped_gt_labels, _ = utils.get_nearest_neighbor_points_with_centroids(seg_label, nearest_n, sampled_db_scan)
cropped_features, sampled_centroids = utils.get_nearest_neighbor_points_with_centroids(output[0], nearest_n, sampled_db_scan)
seg_input = utils.concat_seg_input(cropped_features, cropped_coords, sampled_centroids)

In [79]:
centroids[0].cpu().detach().numpy().T

(15, 3)

In [82]:
sampled_db_scan[0]

[array([0.57430536, 0.2855183 , 0.47671822], dtype=float32),
 array([0.30624685, 0.36208075, 0.48492628], dtype=float32),
 array([0.8211819 , 0.73016703, 0.47616848], dtype=float32),
 array([0.81459004, 0.47073147, 0.45292598], dtype=float32),
 array([0.20956449, 0.53285533, 0.4886102 ], dtype=float32),
 array([0.44301438, 0.2812186 , 0.47983015], dtype=float32),
 array([0.15063447, 0.646658  , 0.4866246 ], dtype=float32),
 array([0.8132192 , 0.87917423, 0.4900948 ], dtype=float32),
 array([0.8185133 , 0.59723413, 0.46267468], dtype=float32),
 array([0.764142  , 0.396976  , 0.47683132], dtype=float32),
 array([0.1243805 , 0.68938357, 0.48455715], dtype=float32),
 array([0.70072955, 0.34770587, 0.47784054], dtype=float32),
 array([0.38049766, 0.3083023 , 0.48709378], dtype=float32),
 array([0.5105428 , 0.2831457 , 0.47171903], dtype=float32),
 array([0.25917622, 0.4368513 , 0.4794978 ], dtype=float32),
 array([0.63389796, 0.30182981, 0.48449236], dtype=float32)]

In [57]:
import models.gen_utils as gen_utils
importlib.reload(gen_utils)
b = gen_utils.cropped_to_global_label_marker(points[0][:3], pred_seg[2], pred_seg[3], nearest_n[0])

In [59]:
np.max(b)

8.0

In [53]:
np.max(b[:,3])

0.0

In [24]:
pred_seg[3].shape

torch.Size([16, 8])

In [7]:
import torch
import models.tsg_utils as utils
import importlib
#두번째를 랜덤아웃풋
for i in range(100):
    sampled_db_scan = utils.dbscan_pc(output[3], output[4], output[5])
    nearest_n = utils.get_nearest_neighbor_idx(output[2], sampled_db_scan, 1024)
    rand_cluster_indexes = [np.random.randint(0,len(nearest_n[i])) for i in range(len(nearest_n))]
    cropped_coords, _ = utils.get_nearest_neighbor_points_hold_batch(output[2], nearest_n, sampled_db_scan, rand_cluster_indexes)
    cropped_gt_labels, _ = utils.get_nearest_neighbor_points_hold_batch(seg_label, nearest_n, sampled_db_scan, rand_cluster_indexes)
    cropped_features, sampled_centroids = utils.get_nearest_neighbor_points_hold_batch(output[0], nearest_n, sampled_db_scan, rand_cluster_indexes)
    seg_input = utils.concat_seg_input(cropped_features, cropped_coords, sampled_centroids)
    
    #테스트
    test_cropped = cropped_coords.cpu().detach().numpy()[0].T
    o3d.visualization.draw_geometries([np_to_pcd(test_cropped, color=[0,1,0]), np_to_pcd(cen_cpu)], mesh_show_back_face=False,mesh_show_wireframe=True)
    test_cropped = cropped_coords.cpu().detach().numpy()[1].T
    o3d.visualization.draw_geometries([np_to_pcd(test_cropped, color=[0,1,0]), np_to_pcd(cen_cpu)], mesh_show_back_face=False,mesh_show_wireframe=True)


NameError: name 'np' is not defined

In [9]:
cropped_coords, _ = utils.get_nearest_neighbor_points_with_centroids(points, nearest_n, sampled_db_scan)

In [13]:
seg_input = cropped_coords

In [14]:
#두번째 아웃풋
with torch.no_grad():
    batch_size = 24
    for batch_start_idx in range(0, seg_input.shape[0], batch_size):
        pred_seg = seg_model(seg_input[batch_start_idx:batch_start_idx+batch_size, :, :])
print(pred_seg[0].shape)

torch.Size([16, 2, 1024])


In [13]:
def sigmoid(x):
    return 1 / (1 +np.exp(-x))
#추론 테스트
for i in range(15):
    test_cropped = cropped_coords.cpu().detach().numpy()[i].T
    test_gt_label = pred_seg[2].cpu().detach().numpy()[i].T.reshape(-1,1)
    test_gt_label = sigmoid(test_gt_label)
    test_gt_label[test_gt_label>0.5] = 1
    test_gt_label[test_gt_label<0.5] = 0
    test_gt_label = test_gt_label.astype(int)
    test_cropped = np.concatenate([test_cropped, test_gt_label],axis=1)
    o3d.visualization.draw_geometries([np_to_by_label(test_cropped, axis=3), np_to_pcd(cen_cpu)], mesh_show_back_face=False,mesh_show_wireframe=True)


KeyboardInterrupt: 

In [24]:
import models.tsg_loss as tsg_loss
importlib.reload(tsg_loss)

<module 'models.tsg_loss' from 'G:\\mesh\\tsegnet\\models\\tsg_loss.py'>

In [14]:
tsg_loss.id_loss(cropped_gt_labels, pred_seg[3])

tensor(2.0783, device='cuda:0')

In [15]:
tsg_loss.first_seg_loss(pred_seg[0], pred_seg[1],cropped_gt_labels)

tensor(6.6404, device='cuda:0')

In [16]:
tsg_loss.second_seg_loss(pred_seg[2], pred_seg[1],cropped_gt_labels)

tensor(18.2075, device='cuda:0')

In [241]:
tsg_loss.distance_loss(distance, xyz, centroid)

tensor(0.1347)

In [14]:
tsg_loss.centroid_loss(offset_xyz, xyz, distance, centroid)

NameError: name 'offset_xyz' is not defined

In [15]:
tsg_loss.chamfer_distance_loss(offset_xyz, xyz, centroid)

NameError: name 'offset_xyz' is not defined

In [4]:
cropped_coord = torch.rand(4,3,2048)
pred_centroid = torch.rand(4,3,1)
gt_centroids_ids = torch.randint(0,7,(4,1,16))
gt_point_labels = torch.randint(0,7,(4,1,2048))
gt_centroids = torch.rand(4,3,16)


In [63]:
test = torch.rand(4,8,3,2048)

In [70]:
(test + test_tt).shape

RuntimeError: The size of tensor a (8) must match the size of tensor b (4) at non-singleton dimension 1

In [71]:
test_tt = torch.rand(4,1, 3,2048)

In [91]:
test_tt = torch.rand(4, 2,2048).min(dim=1)[1]

In [93]:
torch.sum(test_tt)

tensor(4121)

In [76]:
a = np.array([[1,2,3],[4,5,6]])
print(a)
print(a.shape)

[[1 2 3]
 [4 5 6]]
(2, 3)


In [81]:
b = np.array([[5,5,5]])
print(b)
print(b.shape)

[[5 5 5]]
(1, 3)


In [80]:
a+b

array([[ 6,  7,  8],
       [ 9, 10, 11]])

In [47]:
test = test.max(dim=2)[0]

In [49]:
test.shape

torch.Size([4, 8, 16])

In [54]:
torch.nn.functional.smooth_l1_loss(test,test, reduction='none').shape

torch.Size([4, 8, 16])

In [57]:
test_tt = torch.rand(4,8,2,1)

In [58]:
test + test_tt

tensor([[[[1.2729, 0.6645, 0.9892,  ..., 0.8021, 0.6510, 0.9203],
          [0.6358, 1.1128, 1.1635,  ..., 0.7849, 0.6989, 1.3714]],

         [[0.3259, 0.7485, 0.8184,  ..., 0.6446, 1.2563, 0.3431],
          [1.3437, 1.3768, 0.6796,  ..., 1.5759, 0.8211, 0.7183]],

         [[0.6351, 0.7897, 1.3373,  ..., 1.1304, 1.2380, 1.0015],
          [1.6089, 1.8442, 1.8020,  ..., 1.7168, 1.3855, 1.4299]],

         ...,

         [[0.3560, 0.6414, 0.6380,  ..., 0.2264, 0.5644, 0.6045],
          [1.5448, 1.1718, 1.2556,  ..., 1.1649, 1.5193, 1.1950]],

         [[1.1942, 1.3786, 1.7207,  ..., 0.9562, 1.0920, 1.3098],
          [0.9305, 1.3521, 1.1980,  ..., 0.6621, 1.2718, 0.6074]],

         [[1.0234, 0.3031, 0.6626,  ..., 0.5718, 0.3699, 0.1969],
          [1.2610, 1.9084, 1.4566,  ..., 1.1806, 1.5541, 1.1958]]],


        [[[0.8619, 0.2748, 0.4895,  ..., 0.3186, 0.3764, 0.7823],
          [0.8243, 0.4515, 1.0718,  ..., 0.6677, 1.3076, 1.1222]],

         [[1.1553, 1.2519, 1.1542,  ..., 1.22

In [62]:
import models.tsg_utils as utils
import importlib
importlib.reload(utils)
utils.get_cluster_gt_id_by_nearest_gt_centroid(gt_centroids, gt_centroids_ids, pred_centroid, gt_point_labels)

tensor([[ 7],
        [ 6],
        [12],
        [ 0]])


tensor([[2],
        [4],
        [5],
        [6]])

In [60]:
importlib.reload(utils)

utils.get_cluster_gt_id_by_nearest_gt_centroid(gt_centroids, gt_centroids_ids, pred_centroid, gt_point_labels)

tensor([[ 7],
        [ 6],
        [12],
        [ 0]])

In [206]:
centroid_coord = np.load(os.path.join("data","sampled_cls","sugo_2_centroid.npy"))
centroid_exist = np.load(os.path.join("data","sampled_cls","sugo_2_centroid_exist.npy"))
mesh = np.load(os.path.join("data","sampled_cls", "sugo_2_mesh.npy"))

In [202]:
rr = np.array([np.arange(8),np.arange(8)]).T.reshape(16,1)+1

In [207]:
centroid_coord = np.concatenate([centroid_coord, rr], axis=1)
centroid_coord = centroid_coord[centroid_coord[:,0]>-5]

In [176]:
centroid_coord.shape

(16, 4)

In [170]:
mesh

array([[ 0.58299711,  0.16576458,  0.36766305, ..., -0.16167444,
         0.89173833,  2.        ],
       [ 0.58950657,  0.16363464,  0.36518419, ..., -0.08115347,
         0.9654527 ,  2.        ],
       [ 0.59016012,  0.16319963,  0.36911771, ..., -0.07628287,
         0.96889308,  2.        ],
       ...,
       [ 0.08875322,  0.79435488,  0.3449476 , ..., -0.52923539,
        -0.47323164,  0.        ],
       [ 0.08079977,  0.78768028,  0.34755954, ..., -0.47976483,
        -0.69503023,  0.        ],
       [ 0.08543338,  0.78866593,  0.34387204, ..., -0.54302425,
        -0.53451198,  0.        ]])

In [205]:
centroid_exist.shape

(8,)

In [172]:

import models.gen_utils as gen_utils


In [213]:
np.random.rand(8,3)

array([[0.32736046, 0.71189952, 0.20750141],
       [0.56743345, 0.07504726, 0.34684285],
       [0.35949841, 0.4314623 , 0.98791015],
       [0.31151589, 0.44971993, 0.86484811],
       [0.96808667, 0.42096273, 0.95791817],
       [0.64136201, 0.41471365, 0.11085843],
       [0.4789342 , 0.30820783, 0.34425576],
       [0.50173988, 0.38319907, 0.09296238]])

In [187]:
gen_utils.print_3d([gen_utils.np_to_pcd(centroid_coord[:,:3])])

In [195]:
centroid_coord = centroid_coord[centroid_coord[:,0]>-5]

In [196]:
centroid_coord

array([[0.51668505, 0.19062499, 0.3763766 , 0.        ],
       [0.41989021, 0.19416192, 0.39651588, 0.        ],
       [0.61673689, 0.1902163 , 0.37971893, 1.        ],
       [0.32524598, 0.21933546, 0.38427585, 1.        ],
       [0.69463675, 0.21133683, 0.39163248, 2.        ],
       [0.25758288, 0.28703376, 0.38882024, 2.        ],
       [0.76202807, 0.29635131, 0.37765181, 3.        ],
       [0.81273722, 0.40116623, 0.39689242, 4.        ],
       [0.85014329, 0.50730603, 0.4096116 , 5.        ]])

In [208]:
gen_utils.print_3d([gen_utils.np_to_pcd(mesh[:,:3]), gen_utils.np_to_by_label(centroid_coord)])

In [209]:
centroid_coord.shape

(9, 4)